In [ ]:
# Useful starting lines
%matplotlib inline

import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from helpers import *
from cost import *
crossValidation = False
if crossValidation:
    trainDF, testDF = crossValidationSets(0.7)
    testDF.sort_values(['User', 'Movie'], ascending=[1,1], inplace=True)
    trainDF.sort_values(['User', 'Movie'], ascending=[1,1], inplace=True)
else:
    trainDF, testDF = realDataSets() # Kaggle datasets (with no predictions in valuesTest)

print(trainDF.head())
print(testDF.head())

In [ ]:
from predictionAlgorithms import *

print("User mean")
predictionsUserMean = baseline_user_mean(trainDF, testDF)
print("Item mean")
predictionsItemMean = baseline_item_mean(trainDF, testDF)

In [ ]:
print("User mean : test RMSE = {te}"
      .format(te=compute_cost(predictionsUserMean, testDF['Prediction'])))

print("Item mean : test RMSE = {te}"
      .format(te=compute_cost(predictionsItemMean, testDF['Prediction'])))

wU = 0.8
wI = 0.2
print("Mix mean : test RMSE = {te}"
      .format(te=compute_cost((wU*predictionsUserMean + wI*predictionsItemMean) / (wU+wI), testDF['Prediction'])))

In [3]:
from predictionAlgorithms import *

rankALS = 6
numIterationsALS = 20
lambdaALS = 0.07585775750291851

rankALS = 8
lambdaALS = 0.081

predictionsUserMean = baseline_user_mean(trainDF, testDF)
predictionsItemMean = baseline_item_mean(trainDF, testDF)
#predictionsALS = ALSPyspark(trainDF, testDF, rankALS, lambdaALS, numIterationsALS)
predictionsALS = ALSPyspark(trainDF, testDF, rankALS, lambdaALS, numIterationsALS)
print("ALS with bias pywFM")
predictionsALSBias = ALSBias_pywFM(trainDF, testDF)
print("MCMC")
predictionsMCMC = MCMC_pywFM(trainDF, testDF)

if crossValidation:
    print("User mean")
    predictionsUserMeanTrain = baseline_user_median(trainDF, trainDF)
    print("Item mean")
    predictionsItemMeanTrain = baseline_item_median(trainDF, trainDF)
    print("ALS Spark")
    #predictionsALS = ALSPyspark(trainDF, testDF, rankALS, lambdaALS, numIterationsALS)
    predictionsALSTrain = ALSPyspark(trainDF, trainDF, rankALS, lambdaALS, numIterationsALS)
    print("ALS with bias pywFM")
    predictionsALSBiasTrain = ALSBias_pywFM(trainDF, trainDF)
    print("MCMC")
    predictionsMCMCTrain = MCMC_pywFM(trainDF, trainDF)

User mean
Item mean
ALS Spark
ALS with bias pywFM
MCMC


In [ ]:
#print(predictionsALS.Prediction.values)
print(predictionsALS.head())
print(testDF.head())

print(predictionsALSTrain.head())
print(trainDF.head())

In [ ]:
print("User mean : train RMSE = {tr}, test RMSE = {te}"
      .format(tr=compute_cost(predictionsUserMeanTrain, trainDF['Prediction']), te=compute_cost(predictionsUserMean, testDF['Prediction'])))

print("Item mean : train RMSE = {tr}, test RMSE = {te}"
      .format(tr=compute_cost(predictionsItemMeanTrain, trainDF['Prediction']), te=compute_cost(predictionsItemMean, testDF['Prediction'])))

print("ALS Pyspark : train RMSE = {tr}, test RMSE = {te}"
      .format(tr=compute_cost(predictionsALSTrain, trainDF['Prediction']), te=compute_cost(predictionsALS, testDF['Prediction'])))

print("ALS biased : : train RMSE = {tr}, test RMSE = {te}"
      .format(tr=compute_cost(predictionsALSBiasTrain, trainDF['Prediction']), te=compute_cost(predictionsALSBias, testDF['Prediction'])))

print("ALS MCMC : : train RMSE = {tr}, test RMSE = {te}"
      .format(tr=compute_cost(predictionsMCMCTrain, trainDF['Prediction']), te=compute_cost(predictionsMCMC, testDF['Prediction'])))



In [ ]:
print(np.shape(predictionsUserMean[1:10]))
print(np.shape([predictionsUserMean[1:10], predictionsItemMean[1:10]]))
print(predictionsUserMean[1:10])
print(predictionsItemMean[1:10])
print(predictionsALS[1:10])
np.median([predictionsUserMean[1:10], predictionsItemMean[1:10], predictionsALS[1:10]], axis=0)

In [4]:
#wUserMean = 0.0204081632653
#wItemMean = 0.020842379505
#wALS = 1-wUserMean-wItemMean

wUserMean = 0.05
wItemMean = 0.
wALS = 0.2
wALSBias = 0.3
wMCMC = 0.6

#predictions = (wUserMean*predictionsUserMean + wItemMean*predictionsItemMean + wALS*predictionsALS['Prediction']) / (wUserMean + wItemMean + wALS)
predictions = (wUserMean*predictionsUserMean + wItemMean*predictionsItemMean + wALS*predictionsALS +
               wALSBias * predictionsALSBias + wMCMC * predictionsMCMC) / (wUserMean + wItemMean + wALS + wALSBias + wMCMC)
predictionsMedian = np.median([predictionsALS, predictionsALSBias, predictionsMCMC], axis=0)

predictionsTrain = (wUserMean*predictionsUserMeanTrain + wItemMean*predictionsItemMeanTrain + wALS*predictionsALSTrain +
               wALSBias * predictionsALSBiasTrain + wMCMC * predictionsMCMCTrain) / (wUserMean + wItemMean + wALS + wALSBias + wMCMC)
print(predictionsUserMean[0])
print(predictionsItemMean[0])
#print(predictionsALS.Prediction.values[0])
print(predictionsALS[0])
print(predictionsALSBias[0])
print(predictionsMCMC[0])
print(predictions[0])
print(predictionsMedian[0])


print(compute_cost(predictions, testDF['Prediction']))
print(compute_cost(predictionsMedian, testDF['Prediction']))
print(compute_cost(predictionsTrain, trainDF['Prediction']))

NameError: name 'predictionsUserMeanTrain' is not defined

In [ ]:
from cv import *
wUser, wItem, wALS, loss = optimize_weights(predictionsUserMean, predictionsItemMean, predictionsALS._3.values, testDF.Prediction.values)
print(wUser)
print(wItem)
print(wALS)
print(loss)

In [5]:
from helpers import *
make_submission(predictionsMedian)

In [ ]:
predictionsALS

In [ ]:
from predictionAlgorithms import *
predictionsALS = ALSPyspark(trainDF, testDF)

In [ ]:
predictionsALS.columns = ['User', 'Movie', 'Prediction']
predictionsALS.sort_values(['User', 'Movie'], ascending=[1,1], inplace=True)
predictionsALS.head()

In [ ]:
predictions = 0.25*predictionsUserMean+0.25*predictionsItemMean + 0.5*predictionsALS['Prediction']

In [ ]:
make_submission(predictions)